In [1]:
#Import all .csv files for both cities

rootdir = 'dbfs:/autumn_2019/pbchamp/final_project/data'
bikecities = {'WashingtonDC', 'SanFrancisco'}

for bikecity in bikecities:
  
  datapath = rootdir + '/' + bikecity
  
  df = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv(datapath + "/*.csv")
  
  if bikecity == 'WashingtonDC':
    dcdata = df
  elif bikecity == 'SanFrancisco':
    sfdata = df


In [2]:
dcdata.count()

Out[2]: 5839022

In [3]:
sfdata.count()

Out[3]: 4035106

In [4]:
#DC Data Types
dcdata.dtypes

Out[4]: [('Duration', 'int'),
 ('Start date', 'timestamp'),
 ('End date', 'timestamp'),
 ('Start station number', 'int'),
 ('Start station', 'string'),
 ('End station number', 'int'),
 ('End station', 'string'),
 ('Bike number', 'string'),
 ('Member type', 'string')]

In [5]:
#SF Data Types
sfdata.dtypes

Out[5]: [('duration_sec', 'string'),
 ('start_time', 'string'),
 ('end_time', 'string'),
 ('start_station_id', 'string'),
 ('start_station_name', 'string'),
 ('start_station_latitude', 'string'),
 ('start_station_longitude', 'string'),
 ('end_station_id', 'string'),
 ('end_station_name', 'string'),
 ('end_station_latitude', 'string'),
 ('end_station_longitude', 'string'),
 ('bike_id', 'string'),
 ('user_type', 'string'),
 ('member_birth_year', 'string'),
 ('member_gender', 'string'),
 ('bike_share_for_all_trip', 'string')]

In [6]:
#DC Show 5 rows
dcdata.show(5)

+--------+-------------------+-------------------+--------------------+--------------------+------------------+--------------------+-----------+-----------+
Duration| Start date| End date|Start station number| Start station|End station number| End station|Bike number|Member type|
+--------+-------------------+-------------------+--------------------+--------------------+------------------+--------------------+-----------+-----------+
 267|2018-07-01 00:00:58|2018-07-01 00:05:26| 31608|8th & Eye St SE /...| 31610|Eastern Market / ...| W23296| Member|
 530|2018-07-01 00:01:00|2018-07-01 00:09:50| 31230|Metro Center / 12...| 31266| 11th & M St NW| W20838| Member|
 3223|2018-07-01 00:01:07|2018-07-01 00:54:50| 31620| 5th & F St NW| 31289|Henry Bacon Dr & ...| W22733| Member|
 2153|2018-07-01 00:01:13|2018-07-01 00:37:06| 31258| Lincoln Memorial| 31208|M St & New Jersey...| W00196| Casual|
 3214|2018-07-01 00:01:13|2018-07-01 00:54:47| 31620| 5th & F St NW| 31289|Henry Bacon Dr & ...| W21070| Member|
+--------+-------------------+-------------------+--------------------+--------------------+------------------+--------------------+-----------+-----------+
only showing top 5 rows

In [7]:
#SF Show 5 rows
sfdata.show(5)

+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------------------+
duration_sec| start_time| end_time|start_station_id| start_station_name|start_station_latitude|start_station_longitude|end_station_id| end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|bike_share_for_all_trip|
+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------------------+
 53588|2019-03-31 20:59:...|2019-04-01 11:52:...| 377|Fell St at Stanya...| 37.771916882623145| -122.45370447635652| 377|Fell St at Stanya...| 37.771916882623145| -122.45370447635652| 6803|Subscriber| 1980| Other| No|
 74305|2019-03-31 13:56:...|2019-04-01 10:35:...| 385|Woolsey St at Sac...| 37.8505777| -122.2781754| 233|4th Ave at E 12th...| 37.795913| -122.255547| 6415|Subscriber| 1990| Male| No|
 79789|2019-03-31 11:48:...|2019-04-01 09:58:...| 3|Powell St BART St...| 37.78637526861584| -122.40490436553954| 345|Hubbell St at 16t...| 37.766482696439496| -122.39827930927277| 2317| Customer| null| null| No|
 59315|2019-03-31 16:48:...|2019-04-01 09:17:...| 92|Mission Bay Kids ...| 37.772300631747626| -122.39302754402159| 49| S Park St at 3rd St| 37.7807601| -122.3949894| 6631| Customer| 1996| Female| No|
 47886|2019-03-31 19:54:...|2019-04-01 09:12:...| 339|Jackson St at 11t...| 37.80000163118878| -122.26643800735472| 18|Telegraph Ave at ...| 37.85022187449679| -122.26017236709595| 5241|Subscriber| 1974| Female| No|
+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------------------+
only showing top 5 rows

In [8]:
#Rename DC columns to match SF
from pyspark.sql.functions import lit

dcdata = (dcdata
          .withColumnRenamed('Duration', 'duration_sec')
          .withColumnRenamed('Start date', 'start_time')
          .withColumnRenamed('End date', 'end_time')
          .withColumnRenamed('Start station number', 'start_station_id')
          .withColumnRenamed('Start station', 'start_station_name')
          .withColumnRenamed('End station number', 'end_station_id')
          .withColumnRenamed('End station', 'end_station_name')
          .withColumnRenamed('Bike number', 'bike_id')
          .withColumnRenamed('Member type', 'user_type')
          .withColumn('start_station_latitude', lit(''))
          .withColumn('start_station_longitude', lit(''))
          .withColumn('end_station_latitude', lit(''))
          .withColumn('end_station_longitude', lit(''))   
          .withColumn('member_birth_year', lit('')) 
          .withColumn('member_gender', lit(''))
          .withColumn('bike_share_for_all_trip', lit(''))          
         )

In [9]:
#Add 'city' column to DC and populate it with 'DC'
dcdata = dcdata.withColumn('city', lit('DC'))

#Add 'city' column to SF and populate it with 'SF'
sfdata = sfdata.withColumn('city', lit('SF'))

In [10]:
#Union DFs together (by name)
alldata = sfdata.unionByName(dcdata)

In [11]:
alldata.count()

Out[12]: 9874128

In [12]:
alldata.dtypes

Out[16]: [('duration_sec', 'int'),
 ('start_time', 'timestamp'),
 ('end_time', 'timestamp'),
 ('start_station_id', 'string'),
 ('start_station_name', 'string'),
 ('start_station_latitude', 'double'),
 ('start_station_longitude', 'double'),
 ('end_station_id', 'string'),
 ('end_station_name', 'string'),
 ('end_station_latitude', 'double'),
 ('end_station_longitude', 'double'),
 ('bike_id', 'string'),
 ('user_type', 'string'),
 ('member_birth_year', 'string'),
 ('member_gender', 'string'),
 ('bike_share_for_all_trip', 'string'),
 ('city', 'string')]

In [13]:
#Change data type of several columns
alldata = (alldata
          .withColumn('duration_sec', df.duration_sec.cast('integer'))           
          .withColumn('start_time', df.start_time.cast('timestamp'))
          .withColumn('end_time', df.end_time.cast('timestamp'))
          .withColumn('start_station_latitude', df.start_station_latitude.cast('double'))
          .withColumn('start_station_longitude', df.start_station_longitude.cast('double'))
          .withColumn('end_station_latitude', df.end_station_latitude.cast('double'))
          .withColumn('end_station_longitude', df.end_station_longitude.cast('double'))           
          )

In [14]:
#Get count by city
alldata.groupBy(alldata.city).count().show()

+----+-------+
city| count|
+----+-------+
 DC|5839022|
 SF|4035106|
+----+-------+

In [15]:
#Bad data in SF
from pyspark.sql import functions as F
alldata.filter((alldata.city == 'SF') & (F.isnull('start_time'))).count()
#alldata.filter((alldata.city == 'DC') & (F.isnull('start_time'))).count()

#alldata.filter((alldata.city == 'SF') & (F.isnull('end_time'))).count()
#alldata.filter((alldata.city == 'SF') & (F.isnull('start_station_name'))).count()


Out[92]: 258102

In [16]:
#Drop bad data
alldata2 = alldata.dropna(subset='start_time')
alldata2 = alldata2.dropna(subset='start_station_name')

In [17]:
#Get count by city and month
from pyspark.sql.functions import date_trunc
alldata2.groupBy(alldata2.city, date_trunc('mon', alldata2.end_time).alias('month')).count().sort('month', 'city').show()

+----+-------------------+------+
city| month| count|
+----+-------------------+------+
 DC|2018-01-01 00:00:00|168582|
 SF|2018-01-01 00:00:00| 94795|
 DC|2018-02-01 00:00:00|182364|
 SF|2018-02-01 00:00:00|106724|
 DC|2018-03-01 00:00:00|238959|
 SF|2018-03-01 00:00:00|111376|
 DC|2018-04-01 00:00:00|328917|
 SF|2018-04-01 00:00:00|131163|
 DC|2018-05-01 00:00:00|374133|
 SF|2018-05-01 00:00:00|179118|
 DC|2018-06-01 00:00:00|392229|
 SF|2018-06-01 00:00:00|195967|
 DC|2018-07-01 00:00:00|404849|
 SF|2018-07-01 00:00:00|199225|
 DC|2018-08-01 00:00:00|403887|
 SF|2018-08-01 00:00:00|192149|
 DC|2018-09-01 00:00:00|325791|
 SF|2018-09-01 00:00:00|186233|
 DC|2018-10-01 00:00:00|343029|
 SF|2018-10-01 00:00:00|201459|
+----+-------------------+------+
only showing top 20 rows

In [18]:
#Get average duration by city
alldata2.groupBy('city').avg('duration_sec').show()

+----+------------------+
city| avg(duration_sec)|
+----+------------------+
 DC|1120.8009940705824|
 SF| 830.7594242305113|
+----+------------------+

In [19]:
#Get durations longer than one hour
alldata2.filter(alldata2.duration_sec > 3600).count()

Out[135]: 300892

In [20]:
#Get median duration for SF
alldata2.filter(alldata2.city == 'SF').approxQuantile("duration_sec", [0.5], 0.25)

Out[133]: [558.0]

In [21]:
#Get 10 most popular origins in SF
alldata2.filter(alldata2.city == 'SF').groupBy(alldata2.start_station_name).count().sort('count', ascending=False).show(10)

+--------------------+-----+
 start_station_name|count|
+--------------------+-----+
Market St at 10th St|70812|
San Francisco Fer...|69765|
San Francisco Cal...|69175|
 Berry St at 4th St|66684|
San Francisco Cal...|65528|
Montgomery St BAR...|60804|
Powell St BART St...|59002|
The Embarcadero a...|56701|
Steuart St at Mar...|53664|
Howard St at Beal...|50629|
+--------------------+-----+
only showing top 10 rows

In [22]:
#Get 10 most popular destinations in SF
alldata2.filter(alldata2.city == 'SF').groupBy(alldata2.end_station_name).count().sort('count', ascending=False).show(10)

+--------------------+-----+
 end_station_name|count|
+--------------------+-----+
San Francisco Cal...|95231|
San Francisco Fer...|82489|
San Francisco Cal...|79654|
Montgomery St BAR...|73251|
Market St at 10th St|69680|
The Embarcadero a...|66302|
 Berry St at 4th St|64886|
Powell St BART St...|62234|
Steuart St at Mar...|54596|
Powell St BART St...|51075|
+--------------------+-----+
only showing top 10 rows

In [23]:
#Get 10 most popular trips in SF
alldata2\
.filter(alldata2.city == 'SF')\
.groupBy(alldata2.start_station_name, alldata2.end_station_name)\
.count()\
.sort('count', ascending=False)\
.show(10)

+--------------------+--------------------+-----+
 start_station_name| end_station_name|count|
+--------------------+--------------------+-----+
 NULL| NULL|12516|
San Francisco Fer...|The Embarcadero a...|11759|
 Berry St at 4th St|San Francisco Fer...| 8295|
The Embarcadero a...|Steuart St at Mar...| 7845|
Grand Ave at Perk...|19th Street BART ...| 6969|
19th Street BART ...|Grand Ave at Perk...| 6428|
The Embarcadero a...|San Francisco Fer...| 6346|
San Francisco Fer...| Berry St at 4th St| 5641|
Steuart St at Mar...|The Embarcadero a...| 5431|
2nd St at Townsen...|San Francisco Fer...| 5093|
+--------------------+--------------------+-----+
only showing top 10 rows

In [24]:
#Get 10 most popular trips in East Bay (based on longitude)
alldata2\
.filter((alldata2.city == 'SF') & (alldata2.start_station_longitude > -122.36))\
.groupBy('start_station_name', 'end_station_name')\
.count()\
.sort('count', ascending=False)\
.show(10)

+--------------------+--------------------+-----+
 start_station_name| end_station_name|count|
+--------------------+--------------------+-----+
 NULL| NULL|12516|
Grand Ave at Perk...|19th Street BART ...| 6969|
19th Street BART ...|Grand Ave at Perk...| 6428|
 Bay Pl at Vernon St|19th Street BART ...| 4826|
Bancroft Way at C...|Downtown Berkeley...| 4038|
Bancroft Way at C...|Bancroft Way at T...| 3768|
Lake Merritt BART...|2nd Ave at E 18th St| 3651|
19th Street BART ...| Bay Pl at Vernon St| 3480|
Bancroft Way at T...|Downtown Berkeley...| 3309|
 Broadway at 40th St|MacArthur BART St...| 3256|
+--------------------+--------------------+-----+
only showing top 10 rows